### Deliverable 3: Optimize the Model

1. The activation function of hidden layers or output layers are changed for optimization
2. Additional neurons are added to the hidden layers. Additional hidden layers are added
3. Noisy variables are removed from features
4. The model's weights are saved every 5 epochs. The results are saved to an HDF5 file

In [34]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
# Library to suppress warnings or deprecation notes 
import warnings
warnings.filterwarnings('ignore')


In [35]:
# Import and read the charity_data.csv.
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [36]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
#  YOUR CODE GOES HERE
application_df.drop(['EIN','NAME'], axis=1, inplace=True)

In [38]:
# Determine the number of unique values in each column.
#  YOUR CODE GOES HERE
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [39]:
# Look at APPLICATION_TYPE value counts for binning
#  YOUR CODE GOES HERE
application_counts = application_df["APPLICATION_TYPE"].value_counts()

In [40]:
# Determine which values to replace if counts are less than ...?
#  YOUR CODE GOES HERE
replace_application = list(application_counts[application_counts < 500].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [41]:
# Determine which values to replace if counts are less than ..?
#  YOUR CODE GOES HERE
counts = application_df['CLASSIFICATION'].value_counts()
replace_class = application_df.loc[application_df['CLASSIFICATION'].isin(counts[counts.lt(1880)].index), 'CLASSIFICATION']
# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [42]:
# Generate our categorical variable lists
#  YOUR CODE GOES HERE
application_cat = application_df.select_dtypes(include='object').columns.to_list()
application_cat

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [18]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
#  YOUR CODE GOES HERE
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))
# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names_out(application_cat)
encode_df

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34295,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34296,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34297,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [43]:
# Merge one-hot encoded features and drop the originals
#  YOUR CODE GOES HERE
application_df = application_df.merge(encode_df,left_index=True,right_index=True).drop(application_cat,1)


In [44]:
# Split our preprocessed data into our features and target arrays
#  YOUR CODE GOES HERE
y = application_df["IS_SUCCESSFUL"].values.reshape(-1, 1)
X = application_df.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
#  YOUR CODE GOES HERE
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


In [45]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

##  Compile, Train and Evaluate the Model
#### 1. The activation function of hidden layers or output layers are changed for optimization
- using tanh activation

In [132]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30

# Changing Activation Function: try tanh to handle negative inputs in X_train_scaled
nn_tanh = tf.keras.models.Sequential()

# First hidden layer
nn_tanh.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh"))

# Second hidden layer
nn_tanh.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Output layer
nn_tanh.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_tanh.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_36 (Dense)             (None, 80)                2800      
_________________________________________________________________
dense_37 (Dense)             (None, 30)                2430      
_________________________________________________________________
dense_38 (Dense)             (None, 1)                 31        
Total params: 5,261
Trainable params: 5,261
Non-trainable params: 0
_________________________________________________________________


In [133]:
# Compile the model
#  YOUR CODE GOES HERE
nn_tanh.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [134]:
fit_model_tanh = nn_tanh.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100


2022-03-30 16:28:36.845323: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


804/804 [==============================] - 5s 5ms/step - loss: 0.5747 - accuracy: 0.7181
Epoch 2/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5613 - accuracy: 0.7266
Epoch 3/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5583 - accuracy: 0.7272
Epoch 4/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5558 - accuracy: 0.7276
Epoch 5/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5549 - accuracy: 0.7275
Epoch 6/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5542 - accuracy: 0.7285
Epoch 7/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5527 - accuracy: 0.7298
Epoch 8/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5525 - accuracy: 0.7294
Epoch 9/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5512 - accuracy: 0.7296
Epoch 10/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5512 - accuracy: 0.7295
E

In [135]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_tanh.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

2022-03-30 16:37:18.596716: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


268/268 - 1s - loss: 0.5520 - accuracy: 0.7287
Loss: 0.5519989728927612, Accuracy: 0.7287463545799255


## Observation
* With tanh activation and epochs=100 the accuracy is 0.7287463545799255

#### 2- Additional neurons are added to the hidden layers. Additional hidden layers are added

In [138]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  100
hidden_nodes_layer2 = 60
hidden_nodes_layer3 = 30

# Changing Activation Function: try tanh to handle negative inputs in X_train_scaled
nn_tanh2 = tf.keras.models.Sequential()

# First hidden layer
nn_tanh2.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh"))

# Second hidden layer
nn_tanh2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, input_dim=number_input_features, activation="tanh"))

# Third hidden layer
nn_tanh2.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="tanh"))

# Output layer
nn_tanh2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_tanh2.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_41 (Dense)             (None, 100)               3500      
_________________________________________________________________
dense_42 (Dense)             (None, 60)                6060      
_________________________________________________________________
dense_43 (Dense)             (None, 30)                1830      
_________________________________________________________________
dense_44 (Dense)             (None, 1)                 31        
Total params: 11,421
Trainable params: 11,421
Non-trainable params: 0
_________________________________________________________________


In [139]:
# Compile the model
#  YOUR CODE GOES HERE
nn_tanh2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [140]:
fit_model_tanh2 = nn_tanh2.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100


2022-03-30 16:42:37.781854: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


804/804 [==============================] - 7s 8ms/step - loss: 0.5764 - accuracy: 0.7171
Epoch 2/100
804/804 [==============================] - 6s 7ms/step - loss: 0.5606 - accuracy: 0.7260
Epoch 3/100
804/804 [==============================] - 6s 7ms/step - loss: 0.5575 - accuracy: 0.7259
Epoch 4/100
804/804 [==============================] - 5s 6ms/step - loss: 0.5563 - accuracy: 0.7289
Epoch 5/100
804/804 [==============================] - 5s 7ms/step - loss: 0.5551 - accuracy: 0.7282
Epoch 6/100
804/804 [==============================] - 5s 6ms/step - loss: 0.5534 - accuracy: 0.7278
Epoch 7/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5527 - accuracy: 0.7299
Epoch 8/100
804/804 [==============================] - 5s 6ms/step - loss: 0.5515 - accuracy: 0.7293
Epoch 9/100
804/804 [==============================] - 4s 6ms/step - loss: 0.5509 - accuracy: 0.7287
Epoch 10/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5502 - accuracy: 0.7298
E

In [141]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_tanh2.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

2022-03-30 17:18:06.297084: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


268/268 - 1s - loss: 0.5523 - accuracy: 0.7273
Loss: 0.5522618293762207, Accuracy: 0.7273468971252441


## Observation
* With added more hidden layer and tanh activation with epochs=100 the accuracy is 0.7273468971252441
looks like adding more hidden layer did not improve the accuracy

## Feature engineering
#### 3 - Binning more categorical features and removing redundant one, and noisy variables are removed from features.

In [73]:
# Read data
application_df = pd.read_csv("charity_data.csv")

In [74]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
#  YOUR CODE GOES HERE
application_df.drop(['EIN','NAME'], axis=1, inplace=True)

In [75]:
# Determine which values to replace if counts are less than ...?
#  YOUR CODE GOES HERE
replace_application = ['T9','T13','T12','T2','T25','T14','T29','T15','T17']

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Determine which values to replace if counts are less than ..?
#  YOUR CODE GOES HERE
counts = application_df['CLASSIFICATION'].value_counts()
replace_class = application_df.loc[application_df['CLASSIFICATION'].isin(counts[counts.lt(1880)].index), 'CLASSIFICATION']
# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")


C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [76]:
# Generate our categorical variable lists
#  YOUR CODE GOES HERE
application_cat = application_df.select_dtypes(include='object').columns.to_list()
application_cat

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [77]:
# Look at INCOME_AMT value counts for binning
income_counts = application_df["INCOME_AMT"].value_counts()
income_counts

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64

In [86]:
# Determine which values to replace if counts are less than 3000
replace_income = list(income_counts[income_counts < 3300].index)

# Replace in dataframe
for income in replace_income:
    application_df["INCOME_AMT"] = application_df["INCOME_AMT"].replace(income, "Other")
    
# Check to make sure bucketing was successful
application_df["INCOME_AMT"].value_counts()

0                24388
25000-99999       3747
100000-499999     3374
Other             2790
Name: INCOME_AMT, dtype: int64

In [84]:
# Look at AFFILIATION value counts for bucketing
aff_counts = application_df["AFFILIATION"].value_counts()
aff_counts

Independent         18480
CompanySponsored    15705
Family/Parent          64
National               33
Regional               13
Other                   4
Name: AFFILIATION, dtype: int64

In [87]:
# Determine which values to replace if counts are less than 15700
replace_income = list(aff_counts[aff_counts < 15700].index)

# Replace in dataframe
for income in replace_income:
    application_df["AFFILIATION"] = application_df["AFFILIATION"].replace(income, "Other")
    
# Check to make sure bucketing was successful
application_df["AFFILIATION"].value_counts()

Independent         18480
CompanySponsored    15705
Other                 114
Name: AFFILIATION, dtype: int64

In [89]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)

In [90]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df, left_index=True, right_index=True)
application_df = application_df.drop(labels=application_cat, axis=1)

In [91]:
# Drop "SPECIAL_CONSIDERATION_N":
application_df = application_df.drop("SPECIAL_CONSIDERATIONS_N", axis=1)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_100000-499999,INCOME_AMT_25000-99999,INCOME_AMT_Other,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0


In [92]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values.reshape(-1, 1)
X = application_df.drop("IS_SUCCESSFUL", axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [93]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [98]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30

nn_reduced_features = tf.keras.models.Sequential()

# First hidden layer
nn_reduced_features.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn_reduced_features.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn_reduced_features.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_reduced_features.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_24 (Dense)             (None, 80)                2800      
_________________________________________________________________
dense_25 (Dense)             (None, 30)                2430      
_________________________________________________________________
dense_26 (Dense)             (None, 1)                 31        
Total params: 5,261
Trainable params: 5,261
Non-trainable params: 0
_________________________________________________________________


In [105]:
# Compile the model 
nn_reduced_features.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model with 200 epochs
fit_model_nn_reduced_features = nn_reduced_features.fit(X_train_scaled, y_train, epochs=200)

Epoch 1/200


2022-03-30 14:11:08.925475: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


804/804 [==============================] - 6s 6ms/step - loss: 0.5411 - accuracy: 0.7355
Epoch 2/200
804/804 [==============================] - 4s 5ms/step - loss: 0.5402 - accuracy: 0.7358
Epoch 3/200
804/804 [==============================] - 4s 5ms/step - loss: 0.5404 - accuracy: 0.7364
Epoch 4/200
804/804 [==============================] - 4s 6ms/step - loss: 0.5402 - accuracy: 0.7367
Epoch 5/200
804/804 [==============================] - 4s 6ms/step - loss: 0.5403 - accuracy: 0.7371
Epoch 6/200
804/804 [==============================] - 4s 6ms/step - loss: 0.5402 - accuracy: 0.7367
Epoch 7/200
804/804 [==============================] - 4s 5ms/step - loss: 0.5401 - accuracy: 0.7373
Epoch 8/200
804/804 [==============================] - 4s 6ms/step - loss: 0.5396 - accuracy: 0.7360
Epoch 9/200
804/804 [==============================] - 4s 5ms/step - loss: 0.5399 - accuracy: 0.7365
Epoch 10/200
804/804 [==============================] - 4s 5ms/step - loss: 0.5399 - accuracy: 0.7367
E

804/804 [==============================] - 4s 5ms/step - loss: 0.5363 - accuracy: 0.7388
Epoch 161/200
804/804 [==============================] - 4s 5ms/step - loss: 0.5364 - accuracy: 0.7380
Epoch 162/200
804/804 [==============================] - 4s 5ms/step - loss: 0.5361 - accuracy: 0.7386
Epoch 163/200
804/804 [==============================] - 4s 5ms/step - loss: 0.5373 - accuracy: 0.7382
Epoch 164/200
804/804 [==============================] - 4s 5ms/step - loss: 0.5361 - accuracy: 0.7384
Epoch 165/200
804/804 [==============================] - 4s 5ms/step - loss: 0.5362 - accuracy: 0.7386
Epoch 166/200
804/804 [==============================] - 4s 5ms/step - loss: 0.5359 - accuracy: 0.7394
Epoch 167/200
804/804 [==============================] - 4s 6ms/step - loss: 0.5360 - accuracy: 0.7382
Epoch 168/200
804/804 [==============================] - 4s 5ms/step - loss: 0.5365 - accuracy: 0.7381
Epoch 169/200
804/804 [==============================] - 4s 5ms/step - loss: 0.5360 - a

In [111]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_reduced_features.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

2022-03-30 14:29:52.888751: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


268/268 - 1s - loss: 0.5579 - accuracy: 0.7320
Loss: 0.557857871055603, Accuracy: 0.7320116758346558


## Observation
* With the reduced features and relu activation with epochs=200 the accuracy is 0.7309620976448059

#### 4. The model's weights are saved every 5 epochs. The results are saved to an HDF5 file

In [115]:
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints2/",exist_ok=True)
checkpoint_path = "checkpoints2/weights.{epoch:02d}.hdf5"

In [116]:
# Compile the model
nn_reduced_features.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=1000)

# Train the model
fit_model = nn_reduced_features.fit(X_train_scaled,y_train,epochs=200,callbacks=[cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn_reduced_features.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/200
  7/804 [..............................] - ETA: 7s - loss: 0.5619 - accuracy: 0.7188  

2022-03-30 14:30:23.887529: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


804/804 [==============================] - 4s 5ms/step - loss: 0.5407 - accuracy: 0.7387
Epoch 2/200
191/804 [======>.......................] - ETA: 3s - loss: 0.5312 - accuracy: 0.7412
Epoch 00002: saving model to checkpoints2/weights.02.hdf5
804/804 [==============================] - 4s 5ms/step - loss: 0.5358 - accuracy: 0.7388
Epoch 3/200
391/804 [=============>................] - ETA: 2s - loss: 0.5374 - accuracy: 0.7403
Epoch 00003: saving model to checkpoints2/weights.03.hdf5
804/804 [==============================] - 4s 5ms/step - loss: 0.5365 - accuracy: 0.7388
Epoch 4/200
580/804 [====================>.........] - ETA: 1s - loss: 0.5396 - accuracy: 0.7351
Epoch 00004: saving model to checkpoints2/weights.04.hdf5
804/804 [==============================] - 4s 5ms/step - loss: 0.5359 - accuracy: 0.7387
Epoch 5/200
777/804 [===========================>..] - ETA: 0s - loss: 0.5355 - accuracy: 0.7386
Epoch 00005: saving model to checkpoints2/weights.05.hdf5
804/804 [===============

804/804 [==============================] - 5s 6ms/step - loss: 0.5350 - accuracy: 0.7391
Epoch 77/200
804/804 [==============================] - 5s 6ms/step - loss: 0.5346 - accuracy: 0.7390
Epoch 78/200
 84/804 [==>...........................] - ETA: 4s - loss: 0.5410 - accuracy: 0.7314
Epoch 00078: saving model to checkpoints2/weights.78.hdf5
804/804 [==============================] - 4s 5ms/step - loss: 0.5353 - accuracy: 0.7384
Epoch 79/200
288/804 [=========>....................] - ETA: 2s - loss: 0.5362 - accuracy: 0.7411
Epoch 00079: saving model to checkpoints2/weights.79.hdf5
804/804 [==============================] - 4s 5ms/step - loss: 0.5354 - accuracy: 0.7393
Epoch 80/200
478/804 [================>.............] - ETA: 1s - loss: 0.5257 - accuracy: 0.7474
Epoch 00080: saving model to checkpoints2/weights.80.hdf5
804/804 [==============================] - 4s 6ms/step - loss: 0.5351 - accuracy: 0.7390
Epoch 81/200
671/804 [========================>.....] - ETA: 0s - loss: 0.

804/804 [==============================] - 4s 6ms/step - loss: 0.5349 - accuracy: 0.7388
Epoch 115/200
339/804 [===========>..................] - ETA: 2s - loss: 0.5378 - accuracy: 0.7388
Epoch 00115: saving model to checkpoints2/weights.115.hdf5
804/804 [==============================] - 4s 5ms/step - loss: 0.5357 - accuracy: 0.7397
Epoch 116/200
534/804 [==================>...........] - ETA: 1s - loss: 0.5358 - accuracy: 0.7372
Epoch 00116: saving model to checkpoints2/weights.116.hdf5
804/804 [==============================] - 5s 6ms/step - loss: 0.5348 - accuracy: 0.7395
Epoch 117/200
729/804 [==========================>...] - ETA: 0s - loss: 0.5339 - accuracy: 0.7405
Epoch 00117: saving model to checkpoints2/weights.117.hdf5
804/804 [==============================] - 5s 6ms/step - loss: 0.5349 - accuracy: 0.7400
Epoch 118/200
804/804 [==============================] - 5s 6ms/step - loss: 0.5351 - accuracy: 0.7386
Epoch 119/200
120/804 [===>..........................] - ETA: 3s - 

804/804 [==============================] - 5s 6ms/step - loss: 0.5344 - accuracy: 0.7397
Epoch 189/200
804/804 [==============================] - 4s 5ms/step - loss: 0.5343 - accuracy: 0.7392
Epoch 190/200
 35/804 [>.............................] - ETA: 4s - loss: 0.5409 - accuracy: 0.7429
Epoch 00190: saving model to checkpoints2/weights.190.hdf5
804/804 [==============================] - 4s 5ms/step - loss: 0.5344 - accuracy: 0.7397
Epoch 191/200
233/804 [=======>......................] - ETA: 3s - loss: 0.5328 - accuracy: 0.7395
Epoch 00191: saving model to checkpoints2/weights.191.hdf5
804/804 [==============================] - 4s 5ms/step - loss: 0.5346 - accuracy: 0.7389
Epoch 192/200
431/804 [===============>..............] - ETA: 1s - loss: 0.5375 - accuracy: 0.7370
Epoch 00192: saving model to checkpoints2/weights.192.hdf5
804/804 [==============================] - 4s 5ms/step - loss: 0.5345 - accuracy: 0.7385
Epoch 193/200
632/804 [======================>.......] - ETA: 0s - 

2022-03-30 14:45:00.000518: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


268/268 - 1s - loss: 0.5703 - accuracy: 0.7312
Loss: 0.5703132152557373, Accuracy: 0.731195330619812


In [117]:
nn_reduced_features.save("AlphabetSoupCharity_Optimization.h5")